In [1]:
#20220426 水平画角を取得する
import numpy as np
import math
#カメラ-模造紙間の距離 [mm]
d = 1000
#模造紙の実際のサイズ [mm]
Imi = [1091, 788]
#pixel size [pixel]
px = [4000, 3000]
#画像中の模造紙のサイズ [pixel]
Rpx = [3326, 2397]
#中心座標(正規化)
center = [0.5, 0.5]

#画像を丁度埋め尽くすようなサイズ[mm]を求める
Fillx = Imi[0] * px[0] / Rpx[0]
Filly = Imi[1] * px[1] / Rpx[1]
print('x:', Fillx ,'y:', Filly)
#水平画角を求める[deg]
Half_angle = math.degrees(math.atan(Fillx / 2 / d))
Horizontal_angle = 2 * Half_angle
print('Half_angle:', Half_angle)
print('Horizontal_ange:', Horizontal_angle)

x: 1312.086590499098 y: 986.2327909887359
Half_angle: 33.26660900930381
Horizontal_ange: 66.53321801860761


In [28]:
#角度から直線の方程式を求める
#カメラの位置座標
cam1 = np.array([0, 0, 0]) #(x,y,theta)
cam2 = np.array([1.1, 1.1, 90])

def calc_equation(coordinate_bottle, cam: np.ndarray):
    #YOLOの出力結果(x座標)から中心と各ボトルのなす角を計算する
    angle = math.degrees(math.atan(abs(coordinate_bottle - center[0]) / (center[0] / (math.tan(math.radians(Half_angle))))))
    
    if coordinate_bottle <= 0.5:
        slope = math.tan(math.radians(90 + cam[2] + angle))
    else:
        slope = math.tan(math.radians(90 + cam[2] - angle))
                                     
    #切片を求める
    intercept = cam[1] - slope * cam[0]
    return(slope, intercept)

In [29]:
calc_equation(0.490500, np.array([0,0,0]))

(-80.22577065946315, 0.0)

In [30]:
calc_equation(0.191375, np.array([1.1,1.1,90]))

(0.40494272399278436, 0.6545630036079373)

In [32]:
#2直線の交点を求める
def calc_intersection(s1, i1, s2, i2):
    X = (i2 - i1)/(s1 - s2)
    Y = (s1 * i2 - s2 * i1) / (s1 - s2)
    return(X,Y)

In [33]:
calc_intersection(-80.22577065946315, 0.0, 0.40494272399278436, 0.6545630036079373)

(-0.008118035623659044, 0.6512756641490223)

In [34]:
#ボトルの大きさ(高さ)を推定する
def calc_size(X,Y,h):
    #h: camで検出されたボトルの高さ
    
    #Cam1の画像について
    size = Filly * (Y - cam1[1]) * h
    
    '''#Cam2の画像について
    size = Filly * (cam1[1] - X) * h2'''
    return(size)

In [35]:
calc_size(-0.008118035623659044, 0.6512756641490223, 0.373667)

240.00983253230453

In [36]:
def calc_size(X,Y,h2):
    #h: camで検出されたボトルの高さ
    
    '''#Cam1の画像について
    size = Filly * (Y - cam1[1]) * h'''
    
    #Cam2の画像について
    size = Filly * (cam2[0] - X) * h2
    return(size)

In [37]:
calc_size(-0.008118035623659044, 0.6512756641490223, 0.190333)

208.00776833365964

In [ ]:
#尤もらしい組み合わせを判定する
cam1_data = [cam1_A1A2h,cam1_B1A2h,cam1_A1B2h,cam1_B1B2h]
cam2_data = [cam2_A1A2h,cam2_B1A2h,cam2_A1B2h,cam2_B1B2h]

evaluation = []

for i in range(len(cam1_data)):
    ls = []
    ls += [cam1_data[i], cam2_data[i]]
    evaluation.append(np.var(ls))

print(evaluation)
index = np.argsort(evaluation)
print(index)

